In [31]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import copy

# モコズキッチン関連のワードが使われているレシピidを抽出し、csvファイルに書き込む

In [32]:
year = []
for i in range(1998, 2015):
    year.append(i)

col_names_recipe = ["recipe_id", "dish_name", "detail", "background"]
col_names_report = ["recipe_id", "message", "reply"]

まずrecipeデータでもこみち関連のワードを使っているrecipe_idを取得

In [33]:
# 2011年より前のrecipeファイルを読み込む
recipe_before_2011 = pd.DataFrame(columns=col_names_recipe)

for i in range(1998, 2011):
    recipe_temp = pd.read_csv(f"/Users/tamuramasayuki/Desktop/3S/基礎プロジェクトB/programming/dataset/recipe/recipe{i}.csv",
                              names=col_names_recipe, usecols=[0, 2, 3, 6])

    recipe_before_2011 = pd.concat([recipe_before_2011, recipe_temp])

In [34]:
recipe_before_2011.fillna("non-data", inplace=True)

In [35]:
for i in col_names_recipe:
    print(any(recipe_before_2011[i].str.contains('もこみち|モコミチ|もこず|モコズ|Moco\'s|MOCO\'s|moco\'s')))

False
False
False
True


In [52]:
recipe_before_2011.query('background.str.contains("もこみち|モコミチ|もこず|モコズ|Moco\'s|MOCO\'s|moco\'s")', engine='python')

,recipe_id,dish_name,detail,background
122659,c7d3b9a9c037996dd47084a41b9c0a91f8c4152f,薬味丼,玉子かけご飯よりボリューム出るし薬味になる野菜がいっぱいなのですごくさわやかな味✿,香取慎吾くん・もこみちくん・香里奈ちゃんが近所の畑にロケで来た時に食べてたものを再現してみました。


2011年以前はもこみちに関するレシピは存在しない

In [36]:
# 2011年以降のrecipeファイルを読み込む
recipe_from_2011 = pd.DataFrame(columns=col_names_recipe)

for i in range(2011, 2015):
    recipe_temp = pd.read_csv(f"/Users/tamuramasayuki/Desktop/3S/基礎プロジェクトB/programming/dataset/recipe/recipe{i}.csv",
                              names=col_names_recipe, usecols=[0, 2, 3, 6])

    recipe_from_2011 = pd.concat([recipe_from_2011, recipe_temp])

In [37]:
recipe_from_2011.fillna("non-data", inplace=True)

In [38]:
boolean = []
for i in col_names_recipe:
    boolean.append(recipe_from_2011[i].str.contains('もこみち|モコミチ|もこず|モコズ|Moco\'s|MOCO\'s|moco\'s').values)

In [39]:
boolean_index = np.full(len(boolean[0]), False)

In [40]:
# ビット演算でどれか一つでも真のインデックスを取得する
for i in range(len(boolean)):
    boolean_index = boolean_index | boolean[i]

In [41]:
sum(boolean_index)

94

In [66]:
id_recipe = recipe_from_2011["recipe_id"].loc[boolean_index].values

次にreportデータでもこみち関連のワードを使っているrecipe_idを取得

In [42]:
# 2011年より前のreportファイルを読み込む
report_before_2011 = pd.DataFrame(columns=col_names_report)

for i in range(1998, 2011):
    report_temp = pd.read_csv(f"/Users/tamuramasayuki/Desktop/3S/基礎プロジェクトB/programming/dataset/report 2/report{i}.csv",
                              names=col_names_report, usecols=[0, 2, 3])

    report_before_2011 = pd.concat([report_before_2011, report_temp])

In [43]:
report_before_2011.fillna("non-data", inplace=True)

In [44]:
for i in col_names_report:
    print(any(report_before_2011[i].str.contains('もこみち|モコミチ|もこず|モコズ|Moco\'s|MOCO\'s|moco\'s')))

False
True
True


In [53]:
report_before_2011.query('message.str.contains("もこみち|モコミチ|もこず|モコズ|Moco\'s|MOCO\'s|moco\'s")', engine='python')

,recipe_id,message,reply
751733,9c60c4d99bb10e2b9e19b0eaf3563cd3fd3be6fb,"""男子厨房に入るべし！リぴ＊きゅうりは食感を変えて～もこみち風""","""このきゅうりの切り方とっても綺麗❤美味しそうなれぽ感謝です！"""
712634,62749d0af5c14f8c9f068487e96588a6d4a8d79e,"""お友達とのランチはこれで決まり♫♬←もこみち風に（笑）""","""もこみち毎朝見てるから吹き出しても～たわ(笑)ありがとぉ～❤"""


In [54]:
report_before_2011.query('reply.str.contains("もこみち|モコミチ|もこず|モコズ|Moco\'s|MOCO\'s|moco\'s")', engine='python')

,recipe_id,message,reply
556081,9338196a484b326fc70501a8a66827c3e338c37b,"""初のルーを使わないカレーに挑戦しました！このレシピは簡単で嬉しい""","""「みちみち」を「もこみち」と読んじったｙｏ！老眼まっしぐら"""
712634,62749d0af5c14f8c9f068487e96588a6d4a8d79e,"""お友達とのランチはこれで決まり♫♬←もこみち風に（笑）""","""もこみち毎朝見てるから吹き出しても～たわ(笑)ありがとぉ～❤"""
725472,d73e995254e06d8c8f081520bfc9ee610f918ff7,"""オリーブオイルで炒めてイタリアン☆最高に美味しかったです!""","""もこみちさんの様だと思ったらお名前がｗイタリアンありがとう☆"""
853499,cc0ddef1b0311389621bd6a64ef5258b2697d47c,"""さっぱりしていておいしかったです✿これからの季節にぴったり☆""","""でしょでしょ♪夏はこれで決まり！（もこみち風ｗ）れぽ感謝～☆"""
1214295,ebfbaa49746dbb7e6b785cbe606cd98cefdb9630,"""はまってます☆今回は息子のアイデアでにんにくも入れてみました。""","""息子なかなかいいセンスしてるわね～♪末はモコズキッチン2号♡"""
206810,da77138f2824b5e7803db798490b4ce976862b5b,"""リピ☆子供の学校行事のお供に！子供達もお気に入り～♡おいしい☆""","""もっこりおにぎりうまそ＾ｍ＾名付けて「もこみちおにぎり」(笑"""
688068,d5d1b776860fea588c55598e35653b0d8980b5a3,"""次の日もおいしかったです♪また作りまーす(^-^)/""","""れぽお届け有難う～♪目指せもこみちくん応援しちゃう～♡"""
1201915,059e628809ac738e8fb73a7247c095eda0adec10,"""常備の野菜で手軽に一品出来て、さらにごきげんうまい！""","""もこみちですね☆コメントすごく嬉しいです♡作って下さり感謝♪"""


2011年以前のreportデータにもこみち関連のワードは存在するので、考慮する

In [45]:
# 2011年以降のreportファイルを読み込む
report_from_2011 = pd.DataFrame(columns=col_names_report)

for i in range(2011, 2015):
    report_temp = pd.read_csv(f"/Users/tamuramasayuki/Desktop/3S/基礎プロジェクトB/programming/dataset/report 2/report{i}.csv",
                              names=col_names_report, usecols=[0, 2, 3])

    report_from_2011 = pd.concat([report_from_2011, report_temp])

In [46]:
report_from_2011.fillna("non-data", inplace=True)

In [47]:
report_from_2011

,recipe_id,message,reply
0,3efe6df876dd86add7d854451bf7d97417ada480,"""おいしい！写真撮る先からなくなっていきました！""","""ですよね！むむむ！しかもツイスト入れてますね♪おいしそう～♪"""
1,3efe6df876dd86add7d854451bf7d97417ada480,"""ちょっと焼きすぎ…でも香ばしくってめちゃくちゃ美味しかったです！""","""いえいえ！こんがり香ばしそうな焼き上がり♪形がかわい～ですね"""
2,c632a2ada6df36a8185b7b6849e728fe79298a85,"""焼いてる時から良い香り(^ー^* )ﾌﾌ♪とっても美味しかった！""","""おぉ！！おいしそうです～♪つくれぽ、ありがとうございました★"""
3,c632a2ada6df36a8185b7b6849e728fe79298a85,"""簡単でしっとり美味しかったです！バナナの救済レシピになりました♪""","""おいしそうです～！！形も可愛いですね♪"""
4,3efe6df876dd86add7d854451bf7d97417ada480,"""食べ応え抜群！１歳の息子もカリカリカリ・・・""","""小気味よいカリポリ音が聞こえてきそうですね～♪美味しそう！"""
...,...,...,...
337101,439be9adab9f21a65e939eb8be9dfba083da8c4a,"""運動会のお弁当に入れました！子どもにすごく喜ばれました♪""","""くろぶた君もいますね♪ どの子もかわいい～～(*^_^*) """
337102,b0421d4f3303adc1a222f7e1e506e0cc6af8652a,"""優しいお味♡保育園の運動会のお弁当に入れました！美味しかったです""","""運動会シーズンですね♪気に入ってもらえたかなぁ☆また是非 ♡"""
337103,62185a909b32442c1334efbe6693d15b02226869,"""とっても美味しかったです。ニンニクさすが。他の肉でも美味しそう。""","""HandMさん☆作ってくれて嬉しいです♪れぽ有難う☆"""
337104,9a8d7cd82aca73c1c97384346f70c2939e2df0e6,"""アボカド＆キムチ好きにはたまりません!""","""これ美味いっしょ？りぴしてくださいね！Thanks！"""


In [48]:
boolean_report = []
for i in col_names_report:
    boolean_report.append(report_from_2011[i].str.contains('もこみち|モコミチ|もこず|モコズ|Moco\'s|MOCO\'s|moco\'s').values)

In [49]:
boolean_index_report = np.full(len(boolean_report[0]), False)

In [50]:
# ビット演算でどれか一つでも真のインデックスを取得する
for i in range(len(boolean_report)):
    boolean_index_report = boolean_index_report | boolean_report[i]

In [51]:
sum(boolean_index_report)

20

In [86]:
id_report = report_from_2011["recipe_id"].loc[boolean_index_report].values

In [87]:
temp_list = ["62749d0af5c14f8c9f068487e96588a6d4a8d79e",
            "d73e995254e06d8c8f081520bfc9ee610f918ff7",
            "cc0ddef1b0311389621bd6a64ef5258b2697d47c",
            "ebfbaa49746dbb7e6b785cbe606cd98cefdb9630",
            "059e628809ac738e8fb73a7247c095eda0adec10",
            "9c60c4d99bb10e2b9e19b0eaf3563cd3fd3be6fb"
            ]

In [88]:
id_report = np.append(id_report, temp_list)

In [89]:
for i in range(len(id_recipe)):
    for j in range(len(id_report)):
        if id_recipe[i] == id_report[j]:
            print(i, j)

54 16


In [90]:
id_report_temp = np.delete(id_report, 16)

In [91]:
id = np.append(id_recipe, id_report_temp)

In [92]:
len(id)

119

In [102]:
text = np.array([id.T, id_recipe.T, id_report.T])

In [103]:
text

array([array(['21709913597ed302ad157efa4f0b3a6db43a9033',
       '5fc6efd568a1f3f2500e14d2bc0de7b4479e188c',
       'ae662bdc3ae1c62a4a67f951df2d125496f71c4e',
       '5411010461b2f9589e3967bac0b965f1303d4a10',
       '0100c0203c782850d6a367de1de0a738273d61f3',
       '828b90e7bec03a8e9c906a0aa9d9ceeb94b70a16',
       'dc3102cff4033afcb1bf2bc5d1bc3f2ae21b0971',
       'dbb772f5f56aa4616837091b04518e168145a776',
       'a982cf7225001fa29d6dada953d06d1bec556697',
       '6bb989792a5078b734d190aee68058cb4b8c2335',
       'ee11eea1382fc78541d81589211bd6c004754a73',
       '24619b8d7c177f3047e269db1af668ab44a619ce',
       '47eb9bacaa0b05466ea037b7a1f56053045c4921',
       'dc6c3e4836e7a52441f55073a7df5eb42c0108fb',
       'f4fe1eed7fc259991ccdb64889483046f333cd35',
       'e31dd6077131c0bd8e2197a9e501ac05ec2028b2',
       '42ab1e3a333df8c85aed0ab40d83d3007d9f2d5c',
       '515f3a48a1661de0175756135dab59918ecbb74a',
       '92cbd971f8a2b823b385c1fb2c34c9e5528772a9',
       'c41df05308855be0

In [104]:
import csv

In [105]:
with open("moko_index.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(text)